# MODELS

In [3]:
import pandas as pd
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier



In [2]:
df = pd.read_csv('X_TRAIN.csv')

In [3]:
test_df = pd.read_csv('X_TEST.csv')

In [4]:
# poverty = poverty['poverty_score_y'].to_numpy()

In [5]:
df['attended_school'] = df['attended_school'].eq(1)
df = df.drop(columns=['finish_school_age'])

test_df['attended_school'] = test_df['attended_school'].eq(1)
test_df = test_df.drop(columns=['finish_school_age'])


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Models General case

In [4]:
X = df.drop(columns=['uid', 'poverty_score_y'])
X_test = pd.read_csv('X_TEST.csv')

# Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}
stacking_subset = df.sample(n=1300, axis=0)
X_stacking = stacking_subset.reset_index()
uids_stacking = X_stacking['uid']
uids_prediction = X_test['uid']
X_stacking = X_stacking.drop(columns=['uid', 'poverty_score_y'])


NameError: name 'df' is not defined

In [ ]:
reasons = ['not_attend_reason1_2',
       'not_attend_reason1_3', 'not_attend_reason1_4', 'not_attend_reason1_5',
       'not_attend_reason1_6', 'not_attend_reason1_7', 'not_attend_reason1_8',
       'not_attend_reason1_9', 'not_attend_reason1_10',
       'not_attend_reason1_11', 'not_attend_reason1_12',
       'not_attend_reason1_13', 'not_attend_reason1_14',
       'not_attend_reason2_2', 'not_attend_reason2_3', 'not_attend_reason2_4',
       'not_attend_reason2_5', 'not_attend_reason2_6', 'not_attend_reason2_7',
       'not_attend_reason2_8', 'not_attend_reason2_9', 'not_attend_reason2_10',
       'not_attend_reason2_11', 'not_attend_reason2_12',
       'not_attend_reason2_13', 'not_attend_reason2_14', 'school_transportation_time']
X = X.drop(columns=reasons)
X_test = X_test.drop(columns=reasons)
X_stacking = X_stacking.drop(columns=reasons)

In [ ]:
new_cols_to_drop = ['uid', 'finish_school_age']
X_test = X_test.drop(columns=new_cols_to_drop)

In [ ]:
X.columns == X_test.columns

In [ ]:
y = df['poverty_score_y'].to_numpy()

### Random Forest**

In [69]:
param_grid = {
    'n_estimators': [10, 50, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

Best parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 50}
Best cross-validation accuracy: 0.21


# Do it for reals!

In [ ]:
X_stacking = X_stacking.drop(columns=['index'])

In [ ]:
X_stacking

In [ ]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X, y)

# Make predictions on the test data
y_stacking = rf.predict_proba(X_stacking)
y_pred = rf.predict_proba(X_test)



In [ ]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y_pred = pd.DataFrame(y_pred, columns=cols)
y_stacking = pd.DataFrame(y_stacking, columns=cols)

In [ ]:
pred = pd.concat([uids_prediction, y_pred], axis=1)
pred.rename(columns={'uid': 'psu_hh_idcode'}, inplace=True)

stack = pd.concat([uids_stacking, y_stacking], axis=1)
stack.rename(columns={'uid': 'psu_hh_idcode'}, inplace=True)


In [ ]:
pred.to_csv('testset_predictions.csv', index=False)
stack.to_csv('stacking_predictions.csv', index=False)
